In [34]:
import time
from abc import ABC, abstractmethod
from collections import defaultdict
from pandas import DataFrame
from ipywidgets import Text, Button, Label, HBox, Output
from IPython.display import display

In [35]:
class Employee:
    def __init__(self, name: str, salary: int) -> None:
        self.name = name
        self.salary = salary

In [36]:
class EmployeeRepository(ABC):
    @abstractmethod
    def query_employees(self, keyword: str) -> list[Employee]:
        pass

In [37]:
class MemoryEmployeeRepository(EmployeeRepository):
    employees: list[Employee] = [
        Employee('Andy', 9999),
        Employee('Berry', 9999),
        Employee('Anna', 9999),
    ]

    def query_employees(self, keyword: str) -> list[Employee]:
        return [employee for employee in self.employees if keyword.lower() in employee.name.lower()]

In [38]:
class EmployeeService:
    """可以再抽象一层，此处省略"""
    def __init__(self, employee_repository: EmployeeRepository) -> None:
        self._employ_ee_repository = employee_repository

    def query_employees_data(self, keyword: str) -> dict[str, list[str, float]]:
        employees = self._employ_ee_repository.query_employees(keyword)
        result = defaultdict(list)
        for employee in employees:
            result['name'].append(employee.name)
            result['salary'].append(employee.salary)
        return result

In [39]:
class View(ABC):
    @property
    @abstractmethod
    def employee_presenter(self):
        pass

    @employee_presenter.setter
    @abstractmethod
    def employee_presenter(self, presenter):
        # 注入空的东西解决循环依赖
        pass

    @abstractmethod
    def show_employees(self, data: dict[str, list[str, float]]) -> None:
        pass

    @abstractmethod
    def display(self) -> None:
        pass

    @abstractmethod
    def show_status(self) -> None:
        pass

    @abstractmethod
    def hide_status(self) -> None:
        pass

In [40]:
class EmployeeView(View):
    def __init__(self):
        self._employee_presenter: Presenter = None

        self._container: HBox = None
        self._keyword_textbox: Text = None
        self._search_button: Button = None
        self._status: Label = None
        self._output: Output = None
        self._initialize_view()

    def _initialize_view(self) -> None:
        self._keyword_textbox = Text(
            value='',
            placehoder="请输入关键字",
            description="关键字"
        )
        self._search_button = Button(
            description="查询",
            button_style="info",
            tooltip="查询",
            icon="search"
        )
        self._status = Label()
        self._container = HBox(
            (self._keyword_textbox, self._search_button, self._status)
        )
        self._output = Output()
        self._bind_event()

    def _on_click(self, *args):
        keyword = self._keyword_textbox.value
        self._employee_presenter.search(keyword)

    def _bind_event(self):
        self._search_button.on_click(self._on_click)

    @property
    def employee_presenter(self):
        return self._employee_presenter

    @employee_presenter.setter
    def employee_presenter(self, presenter):
        self._employee_presenter = presenter

    def show_employees(self, data: dict[str, list[str, float]]) -> None:
        df = DataFrame(data)
        with self._output:
            self._output.clear_output()
            display(df)

    def display(self) -> None:
        display(self._container, self._output)

    def show_status(self) -> None:
        self._status.value = "正在查询，请稍等..."

    def hide_status(self) -> None:
        self._status.value = ""


In [41]:
class Presenter(ABC):
    @property
    @abstractmethod
    def employ_view(self):
        pass

    @employ_view.setter
    @abstractmethod
    def employ_view(self, view):
        pass

    @abstractmethod
    def search(self, keyword: str) -> None:
        pass

    @abstractmethod
    def display(self) -> None:
        pass


In [42]:
class EmployeePresenter(Presenter):
    def __init__(self, employee_service: EmployeeService) -> None:
        self._employee_service = employee_service
        self._employee_view: View = None

    @property
    def employ_view(self):
        return self._employee_view

    @employ_view.setter
    def employ_view(self, view):
        self._employee_view = view

    def search(self, keyword: str) -> None:
        # 显示进度
        self.employ_view.show_status()
        # 获取数据
        data = self._employee_service.query_employees_data(keyword)
        time.sleep(1)
        # 隐藏进度信息
        self.employ_view.hide_status()
        # 绑定和呈现列表
        self.employ_view.show_employees(data)

    def display(self) -> None:
        self.employ_view.display()

In [43]:
if __name__ == '__main__':
    memory_employee_repository = MemoryEmployeeRepository()
    employee_service = EmployeeService(memory_employee_repository)
    employee_present = EmployeePresenter(employee_service)
    employee_view = EmployeeView()

    employee_present.employ_view = employee_view
    employee_view.employee_presenter = employee_present

    employee_present.display()

Output()